<a href="https://colab.research.google.com/github/hakim3189/MachineLearning/blob/main/Multivariate_Time_Series.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import tensorflow as tf

In [ ]:
df = pd.read_csv('https://drive.google.com/uc?id=1AZRfFoyekqSYpri5183RmJjciRGz_ood', sep=',',
                     infer_datetime_format=True, index_col='datetime', header=0)
df

<ipython-input-33-ab97bed3c6fe>:1: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df = pd.read_csv('https://drive.google.com/uc?id=1AZRfFoyekqSYpri5183RmJjciRGz_ood', sep=',',


,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
datetime,,,,,,,
2006-12-16 17:24:00,4.216,0.418,234.84,18.4,0.0,1.0,17.0
2006-12-16 17:25:00,5.360,0.436,233.63,23.0,0.0,1.0,16.0
2006-12-16 17:26:00,5.374,0.498,233.29,23.0,0.0,2.0,17.0
2006-12-16 17:27:00,5.388,0.502,233.74,23.0,0.0,1.0,17.0
2006-12-16 17:28:00,3.666,0.528,235.68,15.8,0.0,1.0,17.0
...,...,...,...,...,...,...,...
2007-02-14 17:19:00,0.636,0.140,241.16,2.6,0.0,0.0,0.0
2007-02-14 17:20:00,0.552,0.000,240.46,2.2,0.0,0.0,0.0
2007-02-14 17:21:00,0.538,0.000,239.74,2.2,0.0,0.0,0.0


In [ ]:
def normalize_series(data, min, max):
    data = data - min
    data = data / max
    return data
data = df.values
data = normalize_series(data, data.min(axis=0), data.max(axis=0))

In [ ]:
N_FEATURES = len(df.columns)

In [ ]:
SPLIT_TIME = int(len(data) * 0.5)
x_train = data[:SPLIT_TIME]
x_valid = data[SPLIT_TIME:]

In [ ]:
def windowed_dataset(series, batch_size, n_past=24, n_future=24, shift=1):
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(size=n_past + n_future, shift=shift, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(n_past + n_future))
    ds = ds.map(lambda w: (w[:n_past], w[n_past:]))
    return ds.batch(batch_size).prefetch(1)

In [ ]:
BATCH_SIZE = 32
N_PAST = 24
N_FUTURE = 24
SHIFT = 1
# Kode untuk membuat windowed datasets
train_set = windowed_dataset(series=x_train, batch_size=BATCH_SIZE,
                                 n_past=N_PAST, n_future=N_FUTURE,
                                 shift=SHIFT)
valid_set = windowed_dataset(series=x_valid, batch_size=BATCH_SIZE,
                                 n_past=N_PAST, n_future=N_FUTURE,
                                 shift=SHIFT)

In [ ]:
model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(64, input_shape=(N_PAST, N_FEATURES)),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(N_FEATURES)
    ])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
class myCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs={}):
            if (logs.get('mae') < 0.055 and logs.get('val_mae') < 0.055):
                self.model.stop_training = True

callbacks = myCallback()

In [ ]:
# Kode untuk melakukan menyusun struktur sesuai dengan machine learning
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
model.compile(loss='mae',
                  optimizer= optimizer,
                  metrics=["mae"])

In [ ]:
model.fit(train_set,
          validation_data=(valid_set),
          epochs=100,
          callbacks=callbacks,
          verbose=1
    )

Epoch 1/100
   1343/Unknown 14s 9ms/step - loss: 0.0867 - mae: 0.0867

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


1349/1349 ━━━━━━━━━━━━━━━━━━━━ 24s 17ms/step - loss: 0.0866 - mae: 0.0866 - val_loss: 0.0608 - val_mae: 0.0608
Epoch 2/100
1349/1349 ━━━━━━━━━━━━━━━━━━━━ 40s 16ms/step - loss: 0.0599 - mae: 0.0599 - val_loss: 0.0594 - val_mae: 0.0594
Epoch 3/100
1349/1349 ━━━━━━━━━━━━━━━━━━━━ 20s 15ms/step - loss: 0.0593 - mae: 0.0593 - val_loss: 0.0568 - val_mae: 0.0568
Epoch 4/100
1349/1349 ━━━━━━━━━━━━━━━━━━━━ 22s 17ms/step - loss: 0.0582 - mae: 0.0582 - val_loss: 0.0585 - val_mae: 0.0585
Epoch 5/100
1349/1349 ━━━━━━━━━━━━━━━━━━━━ 40s 16ms/step - loss: 0.0580 - mae: 0.0580 - val_loss: 0.0563 - val_mae: 0.0563
Epoch 6/100
1349/1349 ━━━━━━━━━━━━━━━━━━━━ 39s 15ms/step - loss: 0.0575 - mae: 0.0575 - val_loss: 0.0573 - val_mae: 0.0573
Epoch 7/100
1349/1349 ━━━━━━━━━━━━━━━━━━━━ 22s 16ms/step - loss: 0.0575 - mae: 0.0575 - val_loss: 0.0572 - val_mae: 0.0572
Epoch 8/100
1349/1349 ━━━━━━━━━━━━━━━━━━━━ 21s 16ms/step - loss: 0.0575 - mae: 0.0575 - val_loss: 0.0573 - val_mae: 0.0573
Epoch 9/100
1349/1349 ━━━━━━

In [ ]:
import numpy as np
train_pred = model.predict(train_set)
train_pred[0][0]
# Output

1349/1349 ━━━━━━━━━━━━━━━━━━━━ 11s 8ms/step


array([ 0.39024082,  0.24601552,  0.02945043,  0.37300396, -0.01042588,
        0.00500634,  0.86191726], dtype=float32)